<a href="https://colab.research.google.com/github/emerenan/analysing_police_activity/blob/master/police_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd

def readfile(path):
    """ Método para ler os dados do arquivo e retornar um dataframe.
    Args: \n
        path: localização do arquivo que será lido. \n

    Returns: \n
        Dataframe contendo os dados para analise.

    Raises: \n
        ParserError: retorna as linhas que não puderam ser lidas.
    """

    try:
        df = pd.read_csv(path, error_bad_lines=False, warn_bad_lines=True, parse_dates=True, infer_datetime_format=True)
        print(df.head())
        print(df.info())
        print(df.describe())
        return df
    except pd.io.common.ParserError:
        print("Dados não puderam ser lidos")

data = readfile('/content/police.csv')

  state   stop_date stop_time  ...  stop_duration drugs_related_stop district
0    RI  2005-01-04     12:55  ...       0-15 Min              False  Zone X4
1    RI  2005-01-23     23:15  ...       0-15 Min              False  Zone K3
2    RI  2005-02-17     04:15  ...       0-15 Min              False  Zone X4
3    RI  2005-02-20     17:15  ...      16-30 Min              False  Zone X1
4    RI  2005-02-24     01:20  ...       0-15 Min              False  Zone X3

[5 rows x 15 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   state               91741 non-null  object 
 1   stop_date           91741 non-null  object 
 2   stop_time           91741 non-null  object 
 3   county_name         0 non-null      float64
 4   driver_gender       86536 non-null  object 
 5   driver_race         86539 non-null  object 
 6   vio